In [58]:
from PW_explorer.run_clingo import run_clingo
from PW_explorer.load_worlds import load_worlds
from PW_explorer.pwe_helper import rel_slicer
from PW_explorer.time_series import PWETimeSeriesModule
from PW_explorer.pwe_nb_helper import ASPRules
from PW_explorer.export import PWEExport
from PW_explorer.visualize import PWEVisualization

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from nxpd import draw
from nxpd import nxpdParams
nxpdParams['show'] = 'ipynb'
from copy import deepcopy
from collections import OrderedDict
import random
import csv
import numpy as np

In [23]:
def sql2clingo(sqlCommand,originalSchema,colour='black'):
    sql = sqlCommand.replace('"','')
    sqlList = sql.split(' ') 
    #print(sqlList)
    schemaList = originalSchema.split(',')
    #print(schemaList)
    clingoString = 0
    #print(sqlList[0])
    if sqlList[0] == 'UPDATE':
        sql = sql.replace(' = ','=')
        sql = sql.replace(' > ','>')
        sql = sql.replace(' < ','<')
        sql = sql.replace(' >= ','>=')
        sql = sql.replace(' <= ','<=')
        sql = sql.replace(',','')
        sqlList = sql.split(' ') 
        setLoc = sqlList.index('SET')
        whereLoc = sqlList.index('WHERE')
        setCommands = sqlList[setLoc+1:whereLoc]
        setCommands = [x.upper() for x in setCommands]
        whereCommands = sqlList[whereLoc+1:]
        whereCommands = [x.upper() for x in whereCommands if x != 'AND']

        return [whereCommands,setCommands]
    else:
        return 0

In [118]:
def getSetValue(listItem,AWhere,Alocs=['A0','A1','A2','A3','A4'],locator='='):
    for x in listItem:
        for y in range(len(Alocs)):
            if Alocs[y] in x:
                loc1 = x.index(locator) + len(locator)
                num = int(x[loc1:])
                AWhere[y].append(num)
    return AWhere

In [119]:
def getWhereCondition(listItem,AWhere,maxList,minList,Alocs=['A0','A1','A2','A3','A4'],locator='=',locator2='<',locator3 ='>=',locator4 = '>'):
    for x in listItem:
        for y in range(len(Alocs)):
            if Alocs[y] in x:
                if locator in x:
                    if locator4 not in x:
                        loc1 = x.index(locator) + len(locator)
                        num = int(x[loc1:])
                        AWhere[y].extend([num])
                if locator2 in x:
                    loc1 = x.index(locator2) + len(locator2)
                    num = int(x[loc1:])
                    maxList[y].extend([num])
                if locator3 in x:
                    loc1 = x.index(locator3) + len(locator3)
                    num = int(x[loc1:])
                    minVal[y].extend([num])
    #print(AWhere)
    for x in range(len(maxList)):
        if len(maxList[x]) != 0:
            #print(minList[x],maxList[x])
            AWhere[x].extend([y for y in range(minList[x][0],maxList[x][0])])
    return AWhere

In [155]:
testQueryNames = ['/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows10_Nqueries10.csv','/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows10_Nqueries20.csv','/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows10_Nqueries30.csv']
originalSchema = 'id,a0,a1,a2,a3,a4,a5'.upper()

In [156]:
testQueries

'/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows10_Nqueries20.csv'

In [180]:
testQueries = testQueryNames[0]
with open(testQueries,'r') as f:
    data = f.read()
#print(data)    
queries = data.split('\n')
#print(queries)

In [181]:
whereCom = []
setCom = []

for x in queries:
    clingoString = sql2clingo(x,originalSchema)
    if clingoString != 0:
        whereCom.append(clingoString[0])
        setCom.append(clingoString[1])

In [182]:
# fset =set(frozenset(x) for x in whereCom)
# wlst = [list(x) for x in fset]

In [183]:
# fset =set(frozenset(x) for x in setCom)
# slst = [list(x) for x in fset]

In [184]:
wlst = whereCom
slst = setCom

In [185]:
Alocs = ['A0','A1','A2','A3','A4']
AWhere = [[],[],[],[],[]]
for x in wlst:
    for y in x:
        for z in range(len(Alocs)):
            if Alocs[z] in y:
                AWhere[z].append(y)

In [186]:
fullVals = []
for listItem in slst:
    AWhere = [[],[],[],[],[]]
    fullVals.append(getSetValue(listItem,AWhere))

In [187]:
fullCons = []
for listItem in wlst:
    AWhere = [[],[],[],[],[]]
    maxVal = [[],[],[],[],[]]
    minVal = [[],[],[],[],[]]
    fullCons.append(getWhereCondition(listItem,AWhere,maxVal,minVal))

In [188]:
loopConditions = []
loopCon2 = []
for i,val in enumerate(fullVals):
    loop = [[],[],[],[],[]]
    for x in range(len(val)):
        if len(val[x]) != 0:
            for con in range(len(fullCons)):
                if val[x][0] in fullCons[con][x]:
#                     print(x)
#                     print(len(loop))
                    loop[x].append([i,con])
                    loopCon2.append([i,con,x])
    loopConditions.append(loop)

In [189]:
matchers = []
for x in range(len(loopCon2)):
    for y in range(len(loopCon2)):
        loopRef = loopCon2[x]
        loopMatch = [loopCon2[y][1],loopCon2[y][0],loopCon2[y][2]]
        if loopRef == loopMatch:
            print(loopRef,'ref')
            print(loopCon2[y],'Match')
            matchers.append(loopRef)

[1, 1, 1] ref
[1, 1, 1] Match
[1, 8, 1] ref
[8, 1, 1] Match
[8, 1, 1] ref
[1, 8, 1] Match
[8, 8, 1] ref
[8, 8, 1] Match
[17, 17, 2] ref
[17, 17, 2] Match
[17, 25, 2] ref
[25, 17, 2] Match
[25, 17, 2] ref
[17, 25, 2] Match
[25, 25, 2] ref
[25, 25, 2] Match
[33, 33, 4] ref
[33, 33, 4] Match
[33, 40, 4] ref
[40, 33, 4] Match
[40, 33, 4] ref
[33, 40, 4] Match
[40, 40, 4] ref
[40, 40, 4] Match


In [190]:
for x in matchers:
    print(wlst[x[0]],slst[x[1]])
    print(wlst[x[1]],slst[x[0]])
    print('')

['A1>=38', 'A1<48', 'A4>=2', 'A4<12'] ['A1=38', 'A2=21']
['A1>=38', 'A1<48', 'A4>=2', 'A4<12'] ['A1=38', 'A2=21']

['A1>=38', 'A1<48', 'A4>=2', 'A4<12'] ['A1=38', 'A2=21']
['A1>=38', 'A1<48', 'A4>=2', 'A4<12'] ['A1=38', 'A2=21']

['A1>=38', 'A1<48', 'A4>=2', 'A4<12'] ['A1=38', 'A2=21']
['A1>=38', 'A1<48', 'A4>=2', 'A4<12'] ['A1=38', 'A2=21']

['A1>=38', 'A1<48', 'A4>=2', 'A4<12'] ['A1=38', 'A2=21']
['A1>=38', 'A1<48', 'A4>=2', 'A4<12'] ['A1=38', 'A2=21']

['A4>=51', 'A4<61', 'A2>=83', 'A2<93'] ['A2=90', 'A1=46']
['A4>=51', 'A4<61', 'A2>=83', 'A2<93'] ['A2=90', 'A1=46']

['A4>=51', 'A4<61', 'A2>=83', 'A2<93'] ['A2=90', 'A1=46']
['A4>=51', 'A4<61', 'A2>=83', 'A2<93'] ['A2=90', 'A1=46']

['A4>=51', 'A4<61', 'A2>=83', 'A2<93'] ['A2=90', 'A1=46']
['A4>=51', 'A4<61', 'A2>=83', 'A2<93'] ['A2=90', 'A1=46']

['A4>=51', 'A4<61', 'A2>=83', 'A2<93'] ['A2=90', 'A1=46']
['A4>=51', 'A4<61', 'A2>=83', 'A2<93'] ['A2=90', 'A1=46']

['A0>=62', 'A0<72', 'A4>=84', 'A4<94'] ['A0=59', 'A4=91']
['A0>=62', 'A0